In [2]:
import time
import json
import os
import threading
import ctypes
from ctypes import wintypes
import winreg
from pynput import mouse
import win32api
import wmi
import re

In [3]:
# Global variables
DEVICE_TYPE = ''
PREFERENCES = {}
data_buffer = []
stop_logging = False
LOG_FILE = 'mouse_data.json'
MAX_FILE_SIZE = 1 * 1024 * 1024  


In [4]:
def detect_input_devices():
    """Detect input devices using Windows WMI"""
    try:
        c = wmi.WMI()
        
        
        # Get pointing devices (mice, trackpads, etc.)
        pointing_devices = c.Win32_PointingDevice()

        for device in pointing_devices:
            name = device.Name or "Unknown Device"
            device_type = device.PointingType
            
            # Classify device type
            if device_type == 2:
                device_class = "Mouse"
            elif device_type == 3:
                device_class = "Trackball"
            elif device_type == 7:
                device_class = "Trackpad/Touchpad"
            else:
                device_class = f"Unknown Type ({device_type})"
            
            print(f"  - {name}: {device_class}")

        

        # Get Human Interface Devices
        hid_devices = c.Win32_PnPEntity(ConfigManagerErrorCode=0)
        input_devices = []
        
        for device in hid_devices:
            if device.Name and any(keyword in device.Name.lower() for keyword in 
                                 ['mouse', 'trackpad', 'touchpad', 'stylus', 'pen', 'tablet', 'touch']):
                input_devices.append(device.Name)
        
        for device in sorted(set(input_devices)):
            return(classify_device(device))

            
    except ImportError:
        print("WMI module not installed. Install with: pip install WMI")
    except Exception as e:
        print(f"Error: {e}")

def classify_device(device_name):
    """Classify device based on name patterns"""
    name_lower = device_name.lower()
    
    if any(word in name_lower for word in ['stylus', 'pen', 'wacom', 'huion']):
        return "STYLUS/PEN"
    elif any(word in name_lower for word in ['trackpad', 'touchpad', 'synaptics', 'elan']):
        return "TRACKPAD"
    elif 'mouse' in name_lower:
        if any(word in name_lower for word in ['wireless', 'bluetooth']):
            return "WIRELESS MOUSE"
        else:
            return "MOUSE"
    elif any(word in name_lower for word in ['touch', 'finger']):
        return "TOUCHSCREEN"
    else:
        return "OTHER INPUT"

# Function to get user preferences and settings
def get_preferences():
    global PREFERENCES
    try:
        key = winreg.OpenKey(winreg.HKEY_CURRENT_USER, r'Control Panel\Mouse')
        PREFERENCES['MouseSensitivity'] = int(winreg.QueryValueEx(key, 'MouseSensitivity')[0])
        PREFERENCES['MouseSpeed'] = int(winreg.QueryValueEx(key, 'MouseSpeed')[0])
        PREFERENCES['MouseThreshold1'] = int(winreg.QueryValueEx(key, 'MouseThreshold1')[0])
        PREFERENCES['MouseThreshold2'] = int(winreg.QueryValueEx(key, 'MouseThreshold2')[0])
        PREFERENCES['DoubleClickSpeed'] = int(winreg.QueryValueEx(key, 'DoubleClickSpeed')[0])
        PREFERENCES['SwapMouseButtons'] = bool(int(winreg.QueryValueEx(key, 'SwapMouseButtons')[0]))
        winreg.CloseKey(key)
    except Exception as e:
        print(f"Error accessing registry: {e}")





#### Mouse event handlers

In [6]:
# 1. Mouse Movement Trajectories
def on_move(x, y):
    if stop_logging:
        return

    event = {
        'Event': 'Move',
        'Position': {'X': x, 'Y': y},
        'Time': time.time()
    }
    data_buffer.append(event)

# 2. Click Patterns
def on_click(x, y, button, pressed):
    if stop_logging:
        return

    event = {
        'Event': 'Click',
        'Button': str(button),
        'Pressed': pressed,
        'Position': {'X': x, 'Y': y},
        'Time': time.time()
    }
    data_buffer.append(event)

# 3. Scroll Behavior
def on_scroll(x, y, dx, dy):
    if stop_logging:
        return

    event = {
        'Event': 'Scroll',
        'Delta': {'dx': dx, 'dy': dy},
        'Position': {'X': x, 'Y': y},
        'Time': time.time()
    }
    data_buffer.append(event)

# Start monitoring mouse events
def start_monitoring():
    # Start a listener for mouse events
    with mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll) as listener:
        listener.join()



#### Functions for logging

In [8]:

def limit_log_file_size():
    global stop_logging
    try:
        if os.path.exists(LOG_FILE) and os.path.getsize(LOG_FILE) > MAX_FILE_SIZE:
            stop_logging = True
            print("Log file size limit reached. Stopping logging.")
    except Exception as e:
        print(f"Error limiting log file size: {e}")

# Function to save data to JSON file
def save_data():
    limit_log_file_size()
    if stop_logging:
        return

    try:
        with open(LOG_FILE, 'a') as f:
            for entry in data_buffer:
                json.dump(entry, f)
                f.write('\n')
        data_buffer.clear()
    except Exception as e:
        print(f"Error saving data: {e}")

# Function to periodically save data
def periodic_saver(interval=5):
    while not stop_logging:
        time.sleep(interval)
        if data_buffer:
            save_data()

#### Calling all functions 

In [ ]:
# Identify device type
DEVICE_TYPE=detect_input_devices()
print(f"Device Type: {DEVICE_TYPE}")
    
# Get preferences and settings
get_preferences()
print(f"Preferences: {PREFERENCES}")
    
# Initialize log file with device type and preferences
initial_data = {
        'DeviceType': DEVICE_TYPE,
        'Preferences': PREFERENCES,
        'Timestamp': time.time()
    }
with open(LOG_FILE, 'w') as f:
        json.dump(initial_data, f)
        f.write('\n')
    
    # Start periodic saver thread
saver_thread = threading.Thread(target=periodic_saver, daemon=True)
saver_thread.start()
    
# Start monitoring mouse events
start_monitoring()



  - USB Input Device: Mouse
  - HID-compliant mouse: Mouse
Device Type: MOUSE
Preferences: {'MouseSensitivity': 17, 'MouseSpeed': 1, 'MouseThreshold1': 6, 'MouseThreshold2': 10, 'DoubleClickSpeed': 500, 'SwapMouseButtons': False}
Log file size limit reached. Stopping logging.
